In [ ]:
import echoviz as ecv
import h5py
import numpy as np

from pathlib import Path
from scipy.ndimage import binary_fill_holes, grey_closing

from ply.voxelize import *

In [ ]:
def load_inputs(path):
    pdata = path.parent.parent.joinpath(path.stem + ".h5")
    hdf = h5py.File(pdata, 'r')
    # Voxel info
    origin = hdf["VolumeGeometry"]["origin"][()]
    directions = hdf["VolumeGeometry"]["directions"][()]
    spacing = hdf["VolumeGeometry"]["resolution"][()]
    nbf = int(hdf["VolumeGeometry"]["frameNumber"][()])
    # Input frames
    inputs = []
    for i in range(nbf):
        voxels = hdf["CartesianVolume"][f"vol{i + 1:02d}"][()]
        inputs.append(ecv.VoxelGrid(voxels, origin, directions, spacing))
    voxshape = inputs[0].values.shape
    hdf.close()
    return {"nbf": nbf, "voxshape": voxshape, "inputs": inputs}

def load_annotations(pdata, nbf, voxshape, inputs): 
    labels = {"eigen-ant": [], "eigen-post": [], "normal-ant": [], "normal-post": [],
              "rg-ant": [], "rg-post": [], "filter-ant": [], "filter-post": []}
    thickness = 0.004
    div = 3
    for f, afname in zip(range(nbf), pdata.iterdir()):
        i = inputs[f]
        pfname = afname.with_stem(f"posterior-{afname.stem.split('-')[1]}")
        # Oldest way of doing it (extrude along normals with same thickness)
        oant = eigen_extrude(afname, i.spacing, voxshape, i.origin, i.directions, extrude=thickness)
        opost = eigen_extrude(pfname, i.spacing, voxshape, i.origin, i.directions, extrude=thickness)
        labels["eigen-ant"].append(ecv.VoxelGrid(oant, i.origin, i.directions, i.spacing))
        labels["eigen-post"].append(ecv.VoxelGrid(opost, i.origin, i.directions, i.spacing))
        # Current way of doing it (extrude along normals with same thickness)
        oant = normal_extrude(afname, i.spacing, voxshape, i.origin, i.directions, extrude=thickness)
        opost = normal_extrude(pfname, i.spacing, voxshape, i.origin, i.directions, extrude=thickness)
        labels["normal-ant"].append(ecv.VoxelGrid(oant, i.origin, i.directions, i.spacing))
        labels["normal-post"].append(ecv.VoxelGrid(opost, i.origin, i.directions, i.spacing))
        # Select according to voxel intensity
        nant = filter_extrude(afname, i.spacing, voxshape, i.origin, i.directions, i.values, thickness, div)
        npost = filter_extrude(pfname, i.spacing, voxshape, i.origin, i.directions, i.values, thickness, div)
        labels["filter-ant"].append(ecv.VoxelGrid(nant, i.origin, i.directions, i.spacing))
        labels["filter-post"].append(ecv.VoxelGrid(npost, i.origin, i.directions, i.spacing))
        # Select according to subdivided voxel intensity
        snant = region_growing(afname, i.spacing, voxshape, i.origin, i.directions, i.values, thickness, div)
        snpost = region_growing(pfname, i.spacing, voxshape, i.origin, i.directions, i.values, thickness, div)
        labels["rg-ant"].append(ecv.VoxelGrid(snant, i.origin, i.directions, i.spacing))
        labels["rg-post"].append(ecv.VoxelGrid(snpost, i.origin, i.directions, i.spacing))
    return labels

In [ ]:
reviewed = "reviewed"
pdata_FMR = Path(f"~/Documents/data/test-data/{reviewed}/20150624151535_FMR").expanduser()
pdata_FED = Path(f"~/Documents/data/test-data/{reviewed}/20150825110314_FED").expanduser()
# Load inputs frames
fmr = load_inputs(pdata_FMR)
fed = load_inputs(pdata_FED)
# Load annotations
fmr.update(load_annotations(pdata_FMR, fmr["nbf"], fmr["voxshape"], fmr["inputs"]))
fed.update(load_annotations(pdata_FED, fed["nbf"], fed["voxshape"], fed["inputs"]))

In [ ]:
# Plotting
data, name = fed, "FED"
axis, f = 0, int(data["nbf"] / 2)

# Plot one slice (middle)
foo = ecv.plot_slice(data["inputs"][f], {"anterior": data["eigen-ant"][f], "posterior": data["eigen-post"][f]},
                     int(data["voxshape"][axis]/2), axis=axis,
                     title=f"{name} eigen (3mm thickness along normals) frame {f}, clinically {reviewed}")
foo = ecv.plot_slice(data["inputs"][f], {"anterior": data["normal-ant"][f], "posterior": data["normal-post"][f]},
                     int(data["voxshape"][axis]/2), axis=axis,
                     title=f"{name} normal (3mm thickness along normals) frame {f}, clinically {reviewed}")
foo = ecv.plot_slice(data["inputs"][f], {"anterior": data["filter-ant"][f], "posterior": data["filter-post"][f]},
                     int(data["voxshape"][axis]/2), axis=axis,
                     title=f"{name} filter frame {f}, clinically {reviewed}")
foo = ecv.plot_slice(data["inputs"][f], {"anterior": data["rg-ant"][f], "posterior": data["rg-post"][f]},
                     int(data["voxshape"][axis]/2), axis=axis,
                     title=f"{name} region growing frame {f}, clinically {reviewed}")
# Plot full sequence
ecv.sliced_sequence(data["inputs"], {"anterior": data["eigen-ant"], "posterior": data["eigen-post"]},
                    int(data["voxshape"][axis] / 2), axis=axis,
                    title=f"{name} eigen (3mm thickness along normals), clinically {reviewed}",
                    filename=f"{name}_eigenway_axis-{axis}_{reviewed}.gif")
ecv.sliced_sequence(data["inputs"], {"anterior": data["normal-ant"], "posterior": data["normal-post"]},
                    int(data["voxshape"][axis] / 2), axis=axis,
                    title=f"{name} normal (3mm thickness along normals), clinically {reviewed}",
                    filename=f"{name}_normalway_axis-{axis}_{reviewed}.gif")
ecv.sliced_sequence(data["inputs"], {"anterior": data["filter-ant"], "posterior": data["filter-post"]},
                    int(data["voxshape"][axis] / 2), axis=axis,
                    title=f"{name} filter, clinically {reviewed}",
                    filename=f"{name}_filterway_axis-{axis}_{reviewed}.gif")
ecv.sliced_sequence(data["inputs"], {"anterior": data["rg-ant"], "posterior": data["rg-post"]},
                    int(data["voxshape"][axis] / 2), axis=axis,
                    title=f"{name} region growing, clinically {reviewed}",
                    filename=f"{name}_rgway_axis-{axis}_{reviewed}.gif")

In [ ]:
def add_postprocess(data):
    out = {"pp-ant": [], "pp-post": []}
    for f in range(data["nbf"]):
        i = data["inputs"][f]
        ant = grey_closing(data["rg-ant"][f].values, (2, 2, 2), None, None, mode="reflect", cval=0, origin=0)
        post = grey_closing(data["rg-post"][f].values, (2, 2, 2), None, None, mode="reflect", cval=0, origin=0)
        #ant = binary_fill_holes(data["new-ant"][f].values, None, origin=0)
        #post = binary_fill_holes(data["new-post"][f].values, None, origin=0)
        ant = binary_fill_holes(ant, None, origin=0)
        post = binary_fill_holes(post, None, origin=0)
        out["pp-ant"].append(ecv.VoxelGrid(ant, i.origin, i.directions, i.spacing))
        out["pp-post"].append(ecv.VoxelGrid(post, i.origin, i.directions, i.spacing))
    return out

# Do some post-processing to see if it helps
fmr.update(add_postprocess(fmr)), fed.update(add_postprocess(fed))

In [ ]:
# Plotting
data, name = fmr, "FMR"
axis = 0

ecv.sliced_sequence(data["inputs"], {"anterior": data["pp-ant"], "posterior": data["pp-post"]},
                    int(data["voxshape"][axis] / 2), axis=axis,
                    title=f"{name} region growing + post-processed, clinically {reviewed}",
                    filename=f"{name}_post-processed_axis-{axis}_{reviewed}.gif")